# Fine-tuning a gpt-3.5 ReAct Agent on Better Chain of Thought

In this guide, we fine-tune a ReAct Agent powered by gpt-3.5-turbo to perform better chain-of-thought prompting over financial statements.

We do this in the following steps:
1. Setup LlamaIndex query engine tools over Uber 10Q filings.
2. Use our dataset generator to generate a training/evaluation question dataset over a sample 10Q filing. Add complex variations to each question to account for multiple quarters (these complex questions help to induce chain-of-thought prompting).
3. Feed these questions through a GPT-4 ReAct Agent. Log inputs/outputs as a dataset to fine-tune over.
4. Call OpenAI fine-tuning endpoints to fine-tune gpt-3.5-turbo on this dataset.
5. Run qualitative evaluation: show that the fine-tuned model performs better in chain-of-thought prompting than the base model.

#### Note
Each execution of an agent can involve multiple LLM calls through the ReAct chain-of-thought loop. The prompt inputs/output pair for each LLM call is logged as an individual datapoint in the training dataset, in the chat message format.

A big TODO here is to add more quantitative metrics for better evaluation. 

## Setup Data + Build Query Engine Tools

In this section, we load in 3 Uber 10Q fiings (March, June, September). We also setup a standard vector index over each document. This gives the agent the tools to do vector search within any given document.

In [1]:
%pip install llama-index-finetuning
%pip install llama-index-finetuning-callbacks
%pip install llama-index-llms-openai

  Using cached pydantic-1.10.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (150 kB)
Using cached pydantic-1.10.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.3
    Uninstalling pydantic-2.6.3:
      Successfully uninstalled pydantic-2.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
zhipuai 2.0.1 requires pydantic>=2.5.2, but you have pydantic 1.10.14 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement llama-index-finetuning-callbacks (from versions: none)
ERROR: No matching distribution found for llama-index-finetuning-callbacks
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel

In [7]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import (
    OpenAIEmbedding,
)  # pants: no-infer-dep

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [8]:
# one-api代理的glm服务 
llm = OpenAI(model="gpt-4", temperature=0.3, api_key = "sk-ApUK41y73g8qMbrz36A81641752946449f10BbBe32Ff2b7c",api_base="http://localhost:3000/v1")
embeddings = OpenAIEmbedding(api_key = "EMPTY",api_base="http://127.0.0.1:9997/v1")


In [9]:

try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/meas"
    )
    meas_index = load_index_from_storage(storage_context)
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/topo"
    )
    topo_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/geo"
    )
    geo_index = load_index_from_storage(storage_context)
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/asset"
    )
    asset_index = load_index_from_storage(storage_context)
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/psr"
    )
    psr_index = load_index_from_storage(storage_context)
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/base"
    )
    base_index = load_index_from_storage(storage_context)
    index_loaded = True
except:
    index_loaded = False


In [13]:
if not index_loaded:
 
     # load data
    meas_docs = SimpleDirectoryReader(
        input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-测点管理中心.docx"]
    ).load_data()
    topo_docs = SimpleDirectoryReader(
        input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-电网拓扑中心.docx"]
    ).load_data()
    geo_docs = SimpleDirectoryReader(
        input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-电网图形中心.docx"]
    ).load_data()
    asset_docs = SimpleDirectoryReader(
        input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-电网资产中心.docx"]
    ).load_data()
    psr_docs = SimpleDirectoryReader(
        input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-电网资源中心.docx"]
    ).load_data()
    base_docs = SimpleDirectoryReader(
        input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-基础服务中心.docx"]
    ).load_data()

    # build index
    meas_index = VectorStoreIndex.from_documents(
        meas_docs,embed_model=embeddings
    )
    topo_index = VectorStoreIndex.from_documents(
        topo_docs,embed_model=embeddings
    )
    geo_index = VectorStoreIndex.from_documents(
        geo_docs,embed_model=embeddings
    )
    asset_index = VectorStoreIndex.from_documents(
        asset_docs,embed_model=embeddings
    )
    psr_index = VectorStoreIndex.from_documents(
        psr_docs,embed_model=embeddings
    )
    base_index = VectorStoreIndex.from_documents(
        base_docs,embed_model=embeddings
    )
 
    # persist index
    meas_index.storage_context.persist(persist_dir="./storage/meas")
    topo_index.storage_context.persist(persist_dir="./storage/topo")
    geo_index.storage_context.persist(persist_dir="./storage/geo")
    asset_index.storage_context.persist(persist_dir="./storage/asset")
    psr_index.storage_context.persist(persist_dir="./storage/psr")
    base_index.storage_context.persist(persist_dir="./storage/base")

In [14]:
meas_engine = meas_index.as_query_engine(similarity_top_k=3, llm=llm)
topo_engine = topo_index.as_query_engine(similarity_top_k=3, llm=llm)
geo_engine = geo_index.as_query_engine(similarity_top_k=3, llm=llm)
asset_engine = asset_index.as_query_engine(similarity_top_k=3, llm=llm)
psr_engine = psr_index.as_query_engine(similarity_top_k=3, llm=llm)
base_engine = base_index.as_query_engine(similarity_top_k=3, llm=llm)

In [15]:
query_tool_meas = QueryEngineTool.from_defaults(
    query_engine=meas_engine,
    name="meas",
    description=(
        f"关于测点管理中心的业务中台，包括了测量查询服务、事件中心服务、事件类型定义、事件代码表、策略类型、实时开源状态 等业务信息"
        f"eg:检索时带上详细的问题内容"

    ),
)

query_tool_topo = QueryEngineTool.from_defaults(
    query_engine=topo_engine,
    name="topo",
    description=(
        f"关于电网拓扑中心的业务中台，包括了一、电网拓扑中心概述、二、拓扑基础分析服务群、三、拓扑高级分析服务 等业务信息"
        f"eg:检索时带上详细的问题内容"

    ),
)
query_tool_geo= QueryEngineTool.from_defaults(
    query_engine=geo_engine,
    name="geo",
    description=(
        f"关于电网图形中心的业务中台，包括了一、空间查询服务、二、专题图成图服务、三、Gis出图服务、专题图出图服务 等业务信息"
        f"eg:检索时带上详细的问题内容"

    ),
)
query_tool_asset = QueryEngineTool.from_defaults(
    query_engine=asset_engine,
    name="asset",
    description=(
        f"关于电网资产中心的业务中台，包括了资产信息查询服务 业务信息"
        f"eg:检索时带上详细的问题内容"

    ),
)

query_tool_psr = QueryEngineTool.from_defaults(
    query_engine=psr_engine,
    name="psr",
    description=(
        f"关于电网资源中心的业务中台，包括了资源查询服务 业务信息"
        f"eg:检索时带上详细的问题内容"

    ),
)
query_tool_base = QueryEngineTool.from_defaults(
    query_engine=base_engine,
    name="base",
    description=(
        f"关于电基础服务中心的业务中台，包括了3.1电网变更服务、3.2统一认证服务、3.3通用查询服务、3.4结构树服务、3.5权限服务、3.6人员组织查询服务 业务信息"
        f"eg:检索时带上详细的问题内容"

    ),
)
query_engine_tools = [query_tool_meas, query_tool_topo, query_tool_geo, query_tool_asset, query_tool_psr, query_tool_base]

## Setup Base ReAct Agent (gpt-3.5-turbo)

Here we define the baseline ReAct agent over our data, on top of gpt-3.5-turbo.

We run some example queries, and show that the ReAct agent can sometimes enter the incorrect reasoning loop to answer the question.

In [16]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [17]:
llm_agent = OpenAI(model="gpt-4", temperature=0.9, api_key = "sk-ApUK41y73g8qMbrz36A81641752946449f10BbBe32Ff2b7c",api_base="http://localhost:3000/v1") 
base_agent = ReActAgent.from_tools(query_engine_tools, llm=llm_agent, verbose=True)

In [18]:

response = base_agent.chat(
    "查询组织机构的接口是什么"
)
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: base
Action Input: {'input': '查询组织机构的接口是什么'}
Observation: /cispplatform/baseCenter/userOrgService/depts/deptNo
Thought: I can answer without using any more tools.
Answer: /cispplatform/baseCenter/userOrgService/depts/deptNo
/cispplatform/baseCenter/userOrgService/depts/deptNo


## Generate Training/Eval Questions

Generate a synthetic dataset of questions to ask. To do this, we generate an initial set of questions over a "base" document (the March 2022 10Q), and then we use an LLM to generate variations of that question that can apply across multiple quarters. This allows us to more deeply stress-test the LLM reasoning capabilities.


In [20]:
from llama_index.core.evaluation import DatasetGenerator

In [37]:
base_question_gen_query = (
  "你是一名公司业务产品，你的任务是基于系统设计访问的思想设计一套电网拓扑、图形、资源、资产、测点等基础业务。"
  "使用业务接口文档提供的上下文， 制定一些问题，"
  "从上下文中捕捉到重要事实形成问题，"
  "将问题限制在所提供的上下文信息内."
  "**提取的事实需要验证,不需要标注出具体的来源和分类标签**"
  "例如："
  "  停/供电范围分析，在拓扑服务吗？" 
  "  问题" 
  "**请注意你只需要查看接口文档中的业务介绍、请求路径、请求方式，请忽略请求参数和示例信息、返回参数、调用范例**"
)
 
 # load data
meas_docs = SimpleDirectoryReader(
    input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-测点管理中心.docx"]
).load_data()
topo_docs = SimpleDirectoryReader(
    input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-电网拓扑中心.docx"]
).load_data()
geo_docs = SimpleDirectoryReader(
    input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-电网图形中心.docx"]
).load_data()
asset_docs = SimpleDirectoryReader(
    input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-电网资产中心.docx"]
).load_data()
psr_docs = SimpleDirectoryReader(
    input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-电网资源中心.docx"]
).load_data()
base_docs = SimpleDirectoryReader(
    input_files=["/home/dmeck/Documents/电网业务中台/电网资源业务中台接口开发规范说明书V2.01-基础服务中心.docx"]
).load_data()


# 在 
# /llama_index/core/evaluation/dataset_generation.py:246 
# /llama_index/core/evaluation/dataset_generation.py:274
#  增加延时函数
#  await asyncio.sleep(0.5)
dataset_generator = DatasetGenerator.from_documents(
    meas_docs+topo_docs+geo_docs+asset_docs+psr_docs+base_docs,
    question_gen_query=base_question_gen_query,
    llm=llm,
)

/media/gpt4-pdf-chatbot-langchain/pyenv-jsonformer/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:212: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [38]:

import asyncio
import sys
import nest_asyncio
nest_asyncio.apply()


questions = dataset_generator.generate_questions_from_nodes(num=3)

/media/gpt4-pdf-chatbot-langchain/pyenv-jsonformer/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:315: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [39]:
questions

['测点管理中心提供哪些查询量测的服务？', '如何根据单位查找量测？', '如何根据馈线查找量测？']

In [40]:
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate


vary_question_tmpl = """\
你是一位公司产品经理。给定一个关于接口文档的产品需求的问题，你的目标是生成多达 {num_vary} 个问题变体，涉及多个接口文档 。

这可能包括比较/对比不同接口文档，你可以通过业务介绍、请求路径、请求方式生成，或只能通过两个业务介绍的问题（发挥创意！）

你被提供了一组有效的接口文档。请仅生成可以在该组接口文档中回答的问题变体。

For example:
Base Question: 如何通过《OOS文件系统业务介绍》上传文件？
Valid 10Qs: [《组织人员业务接口》, 《任务调度系统接口》, 《OOS文件系统业务介绍》]
Question Variations: 
使用《任务调度系统接口》创建任务后，如何利用《OOS文件系统业务介绍》对任务结果进行存储？
在《组织人员业务接口》中添加新员工后，如何配置《OOS文件系统业务介绍》以分配个人文件存储空间？  
如何结合使用《OOS文件系统业务介绍》和《任务调度系统接口》来优化文件的自动备份流程？  

现在让我们试试吧！

Base Question: {base_question}
Valid 10Qs: {valid_10qs}
Question Variations:
"""


def gen_question_variations(base_questions, num_vary=3):
    """Generate question variations."""

    VALID_10Q_STR = "[关于测点管理中心的业务中台, 关于电网拓扑中心的业务中台, 关于电网图形中心的业务中台, 关于电网资产中心的业务中台, 关于电网资源中心的业务中台, 关于电基础服务中心的业务中台]"

    prompt_tmpl = PromptTemplate(vary_question_tmpl)

    new_questions = []
    for idx, question in enumerate(base_questions):
        new_questions.append(question)
        response = llm.complete(
            prompt_tmpl.format(
                num_vary=num_vary,
                base_question=question,
                valid_10qs=VALID_10Q_STR,
            )
        )
        # parse into newlines
        raw_lines = str(response).split("\n")
        cur_new_questions = [l for l in raw_lines if l != ""]
        print(f"[{idx}] Original Question: {question}")
        print(f"[{idx}] Generated Question Variations: {cur_new_questions}")
        new_questions.extend(cur_new_questions)

    return new_questions


def save_questions(questions, path):
    with open(path, "w") as f:
        for question in questions:
            f.write(question + "\n")


def load_questions(path):
    questions = []
    with open(path, "r") as f:
        for line in f:
            questions.append(line.strip())
    return questions

In [41]:
new_questions = gen_question_variations(questions)

[0] Original Question: 测点管理中心提供哪些查询量测的服务？
[0] Generated Question Variations: ['1. 如何结合使用《关于电网拓扑中心的业务中台》和《关于测点管理中心的业务中台》来获取有关电网设备的实时数据和拓扑信息？', '2. 在《关于电网资源中心的业务中台》中创建新资源后，如何利用《关于测点管理中心的业务中台》监控其性能指标？', '3. 如何通过《关于电网资产中心的业务中台》管理资产信息，并利用《关于测点管理中心的业务中台》获取其相关的量测数据？']
[1] Original Question: 如何根据单位查找量测？
[1] Generated Question Variations: ['1. 如何使用《关于电网资产中心的业务中台》和《关于电网拓扑中心的业务中台》来确定特定变压器的连接状态？', '2. 在《关于电网资源中心的业务中台》中创建新资源后，如何利用《关于电网图形中心的业务中台》可视化其在电网中的位置？', '3. 如何结合使用《关于电基础服务中心的业务中台》和《关于测点管理中心的业务中台》来监控特定区域的用电量并预测需求？']
[2] Original Question: 如何根据馈线查找量测？
[2] Generated Question Variations: ['1. 如何结合《关于电网拓扑中心的业务中台》和《关于电网图形中心的业务中台》来可视化馈线查找量测？', '2. 如何利用《关于电网资产中心的业务中台》和《关于电网资源中心的业务中台》来分析馈线查找量测与资产和资源之间的关系？', '3. 如何使用《关于电基础服务中心的业务中台》和《关于电网拓扑中心的业务中台》来创建基于馈线查找量测的告警和通知系统？']


In [42]:
len(new_questions)

12

In [43]:
train_questions, eval_questions = new_questions[:60], new_questions[60:]

In [44]:
save_questions(train_questions, "train_questions_10q.txt")
save_questions(eval_questions, "eval_questions_10q.txt")

In [45]:
train_questions = load_questions("train_questions_10q.txt")
eval_questions = load_questions("eval_questions_10q.txt")

## Use GPT-4 to Log Input/Output Pairs

We run the train questions through a GPT-4 powered ReAct agent to collect prompt outputs.

Every prompt call to the LLM is logged as an input/output pair. Since the ReAct loop can call the LLM multiple times, this means that multiple input/output pairs may be logged per user query.

Our `OpenAIFineTuningHandler` automatically collects prompt input/outputs when agent queries are run. This dataset can then be saved, in a dataset format `.jsonl` that you can directly feed to the OpenAI Finetuning endpoints.

In [46]:
from llama_index.llms.openai import OpenAI
from llama_index.finetuning.callbacks import OpenAIFineTuningHandler
from llama_index.core.callbacks import CallbackManager
from llama_index.core.agent import ReActAgent

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

from llama_index.core import Settings

# limit the context window artifically to test refine process
Settings.context_window = 2048

In [47]:
agent_llm = OpenAI(model="gpt-4", temperature=0.1, api_key="sk-ApUK41y73g8qMbrz36A81641752946449f10BbBe32Ff2b7c",
                       api_base="http://localhost:3000/v1")
gpt4_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=agent_llm,
    callback_manager=callback_manager,
    verbose=True,
)

In [48]:
for idx, question in enumerate(train_questions):
    print(f"[{idx}] Question: {question}")
    response = gpt4_agent.query(question)
    print(f"[{idx}] Agent Response: {str(response)}")

[0] Question: 测点管理中心提供哪些查询量测的服务？
Thought: I need to use a tool to help me answer the question.
Action: meas
Action Input: {'input': '测点管理中心提供哪些查询量测的服务？'}
Observation: 根据单位查找量测、根据馈线查找量测、根据站所查找量测、根据台区查找量测、根据被测设备查找量测、根据资源ID查找量测、根据资产ID查找量测、根据测量点查找量测、根据县局查找量测点台帐
Thought: I can answer without using any more tools.
Answer: 测点管理中心提供根据单位、馈线、站所、台区、被测设备、资源ID、资产ID、测量点、县局查找量测的服务。
[0] Agent Response: 测点管理中心提供根据单位、馈线、站所、台区、被测设备、资源ID、资产ID、测量点、县局查找量测的服务。
[1] Question: 1. 如何结合使用《关于电网拓扑中心的业务中台》和《关于测点管理中心的业务中台》来获取有关电网设备的实时数据和拓扑信息？
Thought: I need to use a tool to help me answer the question.
Action: topo
Action Input: {"input": "如何获取电网设备的拓扑信息？"}
Observation: {"output": "电网拓扑中心提供了电网设备的拓扑信息，包括设备之间的连接关系、设备的地理位置等。"}
Thought: I need to use a tool to help me answer the question.
Action: meas
Action Input: {"input": "如何获取电网设备的实时数据？"}
Observation: {"output": "测点管理中心提供了电网设备的实时数据，包括设备的运行状态、测量值等。"}
Thought: I can answer without using any more tools.
Answer: 可以通过结合使用《关于电网拓扑中心的业务中台》和《关于测点管理中心的业务中台》来获取有关电网设备的实时数据和拓扑信息。

In [49]:
# save events
finetuning_handler.save_finetuning_events("finetuning_events_10q.jsonl")

Wrote 21 examples to finetuning_events_10q.jsonl


## Create `OpenAIFinetuneEngine`

We create an `OpenAIFinetuneEngine`: the finetune engine will launch a finetuning job, and returning an LLM model that you can directly plugin to the rest of LlamaIndex workflows.

In [ ]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "finetuning_events_10q.jsonl",
    # start_job_id="<start-job-id>"  # if you have an existing job, can specify id here
)

In [ ]:
finetune_engine.finetune()

Num examples: 184
First example:
{'role': 'system', 'content': '\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Tools\nYou have access to a wide variety of tools. You are responsible for using\nthe tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools\nto complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: march_2022\nTool Description: Provides information about Uber quarterly financials ending March 2022\nTool Args: {\'title\': \'DefaultToolFnSchema\', \'description\': \'Default tool function Schema.\', \'type\': \'object\', \'properties\': {\'input\': {\'title\': \'Input\', \'type\': \'string\'}}, \'required\': [\'input\']}\n\n> Tool Name: june_2022\nTool Description: Provides information about Uber quarterly financials ending June 2022\nTool Args: {\'title\': \'DefaultToolF

In [ ]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-OSUTIOyII1IwocEIB2ktcZhB at 0x2ba6868e0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-OSUTIOyII1IwocEIB2ktcZhB",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693700082,
  "finished_at": 1693700955,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:llamaindex::7uVHHzp7",
  "organization_id": "org-1ZDAvajC6v2ZtAP9hLEIsXRz",
  "result_files": [
    "file-rVuUfjj05GUQbWmnth2JT6W9"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-eUSkAcjIXOOSEtPRhSRR6qzb",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 449481
}

In [ ]:
ft_llm = finetune_engine.get_finetuned_model(temperature=0.3)

## Run Some Queries! (Compare Finetuned Agent vs. Base Agent)

We run some sample queries from the evaluation dataset over both our finetuned agent as well as the base agent.

We qualitatively look at their abilities to perform chain of thought prompting in order to arrive at the right answer.

**NOTE**: There's a big TODO to setup quantitative metrics so we can more rigorously evaluate the quality of any agent over an evaluation dataset! 

In [ ]:
ft_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=ft_llm,
    callback_manager=callback_manager,
    verbose=True,
)

In [ ]:
eval_questions = []
with open("eval_questions_10q.txt", "r") as f:
    for line in f:
        eval_questions.append(line.strip())

In [ ]:
# try a sample question
qidx = 0
print(eval_questions[qidx])

What is the total fair value of Uber's financial assets as of March 31, 2022?


In [ ]:
base_response = base_agent.query(eval_questions[qidx])
print(str(base_response))

Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'financial_assets'}
Observation: The financial assets mentioned in the context include non-marketable equity securities, marketable equity securities, and a note receivable from a related party. These assets are measured at fair value on a recurring basis and are categorized into three levels of the fair value hierarchy: Level 1, Level 2, and Level 3. Level 1 assets are valued based on quoted market prices, Level 2 assets are valued using readily available pricing sources or models with market observable inputs, and Level 3 assets are valued based on unobservable inputs and estimation techniques. The Level 3 assets primarily consist of non-marketable equity securities and the note receivable from a related party. The fair value of these assets is estimated using various valuation techniques, including the guideline public company approach and option-pricing models. The fair value of

In [ ]:
ft_response = ft_agent.query(eval_questions[qidx])
print(str(ft_response))

Thought: I need to use the march_2022 tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'financial assets fair value'}
Observation: The financial assets of the company are measured at fair value on a recurring basis. These financial assets include non-marketable equity securities, marketable equity securities, and a note receivable from a related party. The fair value of these assets is determined based on the three-tier fair value hierarchy, with Level 1 valuations based on quoted market prices, Level 2 valuations obtained from readily available pricing sources or models using market observable inputs, and Level 3 valuations based on unobservable inputs and estimation techniques. The company did not make any transfers between the levels of the fair value hierarchy during the reporting period.
Thought: The tool didn't provide the specific fair value of Uber's financial assets as of March 31, 2022. I need to try again.
Action: march_2022
Action Input: {'inp

In [ ]:
# try the original question that failed
test_q = (
    "Can you tell me about the risk factors in the quarter with the highest"
    " revenue growth?"
)
base_response = base_agent.query(test_q)
print(str(base_response))

Thought: I need to find the quarter with the highest revenue growth and then analyze the risk factors for that quarter.
Action: march_2022
Action Input: {'input': 'revenue_growth'}
Observation: The revenue growth for the company in the three months ended March 31, 2022, compared to the same period in 2021, was 136%. This increase in revenue was primarily driven by an increase in Gross Bookings of 35%, primarily due to increases in Mobility Trip volumes as the business recovers from the impacts of COVID-19, and a $1.5 billion increase in Freight revenue resulting from the acquisition of Transplace. Additionally, there was a $304 million increase in Delivery revenue and a $200 million increase in Mobility revenue due to business model changes in the UK.
Thought: The quarter with the highest revenue growth is the quarter ending March 2022. Now I need to analyze the risk factors for that quarter.
Action: march_2022
Action Input: {'input': 'risk_factors'}
Observation: The company faces vari

In [ ]:
# NOTE: this successfully looks at each quarter for revenue growth but still falls behind GPT-4
ft_response = ft_agent.query(test_q)
print(str(ft_response))

Thought: I need to use the tools to get the revenue growth data for each quarter.
Action: march_2022
Action Input: {'input': 'revenue growth'}
Observation: The revenue for the company has increased significantly compared to the same period last year. The revenue growth is primarily driven by an increase in Gross Bookings, particularly in Mobility Trip volumes as the business recovers from the impacts of COVID-19. Additionally, the acquisition of Transplace has contributed to an increase in Freight revenue. The company has also seen an increase in Delivery revenue and Mobility revenue due to certain business model changes. Overall, the revenue growth for the company has been positive.
Thought: Now I have the revenue growth information for the first quarter. I need to compare it with the revenue growth of the second and third quarters to determine which quarter had the highest growth.
Action: june_2022
Action Input: {'input': 'revenue growth'}
Observation: The context information does no

**Observations**: The finetuned model does much better than the base model in terms of reasoning about the current sequence of steps. It passes more detailed answers to the downstream tools and is more capable of refining its approach when initial queries don't work. This applies even if the answer isn't actually found within the context (which is a function of our automatic dataset generation capabilities). 